<a href="https://colab.research.google.com/github/dakilaledesma/herbarium_model_testing/blob/main/2x3xnet/H2022_GPU_DualNet_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! unzip -q /content/drive/MyDrive/UNC/H2022/RCNN_PP/pls224p56_Orchidaceae_test.zip -d /content/
! unzip -q /content/drive/MyDrive/UNC/H2022/orchidaceae_test.zip -d /content/

In [ ]:
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, preprocess_input
from tensorflow.keras.preprocessing import image
from tqdm.notebook import tqdm
import numpy as np
from tensorflow.keras.models import Model, load_model


In [ ]:
model = load_model("/content/drive/MyDrive/UNC/H2022/DualNet1")

In [ ]:
from glob import glob
import os
cat_to_int = {}
for i, f in enumerate(sorted(glob("pls224p56_Orchidaceae_test/*"))):
  cat = os.path.basename(f)
  cat_to_int[cat] = i

In [ ]:
print(cat_to_int.items())

dict_items([('00098', 0), ('00099', 1), ('00100', 2), ('00915', 3), ('01032', 4), ('01306', 5), ('01819', 6), ('01854', 7), ('01963', 8), ('01964', 9), ('02127', 10), ('02133', 11), ('02134', 12), ('02135', 13), ('02136', 14), ('02137', 15), ('02138', 16), ('02246', 17), ('02247', 18), ('02394', 19), ('02395', 20), ('02396', 21), ('02397', 22), ('02398', 23), ('02417', 24), ('02470', 25), ('02471', 26), ('02472', 27), ('02473', 28), ('02474', 29), ('03139', 30), ('03140', 31), ('03141', 32), ('03142', 33), ('03143', 34), ('03144', 35), ('03145', 36), ('03155', 37), ('03156', 38), ('03157', 39), ('03158', 40), ('03159', 41), ('03160', 42), ('03161', 43), ('03162', 44), ('03163', 45), ('03258', 46), ('03658', 47), ('03766', 48), ('03775', 49), ('03776', 50), ('03852', 51), ('03889', 52), ('03890', 53), ('03891', 54), ('03892', 55), ('03893', 56), ('03894', 57), ('03984', 58), ('03994', 59), ('04019', 60), ('04020', 61), ('04326', 62), ('04327', 63), ('04329', 64), ('04490', 65), ('04491'

In [ ]:
valid_fn_t = {}
for fn in sorted(glob("pls224p56_Orchidaceae_test/**/*.*", recursive=True)):
  bn = os.path.basename(fn)
  valid_fn_t[bn] = fn

o_t = []
pl_t = []
y_t = []
orc_t = sorted(glob("orchidaceae_test/**/*.*", recursive=True))
for fn in tqdm(orc_t, total=len(list(orc_t))):
  try:
    bn =  os.path.basename(fn)
    cat = cat_to_int[fn.split("/")[-2]]
    try:
      pl_fn = valid_fn_t[bn]
    except KeyError:
      pass

    img = image.load_img(fn, target_size=(224, 224))
    x = image.img_to_array(img)
    x = preprocess_input(x)
    o_t.append(x)

    img = image.load_img(pl_fn, target_size=(224, 224))
    x = image.img_to_array(img)
    x = preprocess_input(x)
    pl_t.append(x)

    y_t.append(cat)
  except:
    pass


  0%|          | 0/4577 [00:00<?, ?it/s]

In [ ]:
preds = []
for ot, plt in tqdm(zip(o_t, pl_t), total=len(o_t)):
  ot = np.array(ot).reshape((-1, 224, 224, 3))
  plt = np.array(plt).reshape((-1, 224, 224, 3))
  pred = np.argmax(model([ot, plt])[0])
  preds.append(pred)

  0%|          | 0/4577 [00:00<?, ?it/s]

In [ ]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_t, preds))

0.16255188988420363
